In [12]:
import setup
setup.init_django()

In [13]:
from market.models import StockQuote

In [14]:
from django.db.models import Avg, F, RowRange, Window, Max, Min
from django.db.models.functions import TruncDate, FirstValue
from django.utils import timezone
from datetime import timedelta
from decimal import Decimal

In [15]:
days_ago = 30
now = timezone.now()
start_date = now - timedelta(days=30)
end_date = now
ticker = "AAPL"

In [16]:
latest_daily_timestamps = (
    StockQuote.timescale.filter(company__ticker = ticker, time__range=(start_date - timedelta(days=40), end_date))
    .time_bucket('time', '1 day')
    .annotate(date=TruncDate('time'))
    .values('company', 'date')
    .annotate(latest_time=Max('time'))
    .values('company', 'date', 'latest_time')
    .order_by('date') # skipping over dates not in database
)

# timestamps for final of the day
actual_timestamps = list(set([x['latest_time'] for x in latest_daily_timestamps]))
len(actual_timestamps)

53

In [39]:
qs = StockQuote.objects.filter(
    company__ticker = ticker, 
    time__range=(start_date, end_date),
    time__in=actual_timestamps
)

In [ ]:
def get_volume (queryset, days=5):
    # Analyze recent volume trends
    start = -(days-1)
    data = queryset.annotate(
        avg_volume=Window(
            expression = Avg('volume'),
            order_by=F('time').asc(),
            partition_by=[],
            frame = RowRange(start=start, end=0)
        )
    ).order_by('-time').first()

    if not data:
        return None

    volume_change = ((data.volume - data.avg_volume) /
                    data.avg_volume) * 100

    return {
        'avg_volume': float(data.avg_volume),
        'latest_volume': int(data.volume),
        'volume_change_percent': float(volume_change)
    }

In [40]:
get_volume_change(qs)

In [41]:
for obj in ma_val[:5]:
    print(obj.id, obj.close_price, obj.ma, obj.time)

330238 235.5300 231.8323400000000000 2024-11-27 00:55:00+00:00
330226 235.4617 230.4683400000000000 2024-11-26 23:55:00+00:00
330053 229.9500 228.9760000000000000 2024-11-23 00:55:00+00:00
330046 229.9899 228.5560000000000000 2024-11-22 23:55:00+00:00
329863 228.2301 227.6100200000000000 2024-11-21 23:55:00+00:00


In [42]:
frame_start = -(count - 1)
ma_vals = qs.annotate(
        ma_5 = Window( # 5 minute intervals of 5
            expression=Avg('close_price'),
            order_by=F('time').asc(),
            partition_by=[],
            frame = RowRange(start=-4, end=0), # based on amount we want to get
        ),
        ma_20 = Window( # 20 minute intervals of 5
            expression=Avg('close_price'),
            order_by=F('time').asc(),
            partition_by=[],
            frame = RowRange(start=-19, end=0), # based on amount we want to get
        )
).order_by('-time').first()

ma_vals.id, ma_vals.close_price, ma_vals.ma_5, ma_vals.ma_20

(330238,
 Decimal('235.5300'),
 Decimal('231.8323400000000000'),
 Decimal('227.1750850000000000'))